# GTC 2018 Numba Tutorial Notebook 4: Writing CUDA Kernels

## The CUDA Programming Model

Ufuncs (and generalized ufuncs mentioned in the bonus notebook at the end of the tutorial) are the easiest way in Numba to use the GPU, and present an abstraction that requires minimal understanding of the CUDA programming model.  However, not all functions can be written as ufuncs.  Many problems require greater flexibility, in which case you want to write a *CUDA kernel*, the topic of this notebook. 

Fully explaining the CUDA programming model is beyond the scope of this tutorial.  We highly recommend that everyone writing CUDA kernels with Numba take the time to read Chapters 1 and 2 of the CUDA C Programming Guide:

 * Introduction: http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#introduction
 * Programming Model: http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#programming-model

The programming model chapter gets a little in to C specifics, but familiarity with CUDA C can help write better CUDA kernels in Python.

For the purposes of this tutorial, the most important thing is to understand this diagram:
![Thread Hierarchy](http://docs.nvidia.com/cuda/cuda-c-programming-guide/graphics/grid-of-thread-blocks.png "Thread Hierarchy (from CUDA C Programming Guide)")

We will be writing a *kernel* that decribes the execution of a single thread in this hierarchy.  The CUDA compiler and driver will execute our kernel across a *thread grid* that is divided into *blocks* of threads.  Threads within the same block can exchange data very easily during the execution of a kernel, whereas threads in different blocks should generally not communicate with each other (with a few exceptions).

Deciding the best size for the CUDA thread grid is a complex problem (and depends on both the algorithm and the specific GPU compute capability), but here are some very rough heuristics that we follow:

  * the size of a block should be a multiple of 32 threads, with typical block sizes between 128 and 512 threads per block.
  * the size of the grid should ensure the full GPU is utilized where possible.  Launching a grid where the number of blocks is 2x-4x the number of "multiprocessors" on the GPU is a good starting place.  Something in the range of 20 - 100 blocks is usually a good starting point.
  * The CUDA kernel launch overhead does depend on the number of blocks, so we find it best not to launch a grid where the number of threads equals the number of input elements when the input size is very big.  We'll show a pattern for dealing with large inputs below.

Each thread distinguishes itself from the other threads using its unique thread (`threadIdx`) and block (`blockIdx`) index values, which can be multidimensional if launched that way.

## A First Example

This all will be a little overwhelming at first, so let's start with a concrete example.  Let's write our addition function for 1D NumPy arrays.  CUDA kernels are compiled using the `numba.cuda.jit` decorator (not to be confused with the `numba.jit` decorator for the CPU):

In [1]:
from numba import cuda

@cuda.jit
def add_kernel(x, y, out):
    tx = cuda.threadIdx.x # this is the unique thread ID within a 1D block
    ty = cuda.blockIdx.x  # Similarly, this is the unique block ID within the 1D grid

    block_size = cuda.blockDim.x  # number of threads per block
    grid_size = cuda.gridDim.x    # number of blocks in the grid
    
    start = tx + ty * block_size
    stride = block_size * grid_size

    # assuming x and y inputs are same length
    for i in range(start, x.shape[0], stride):
        out[i] = x[i] + y[i]

That's a lot more typing than our ufunc example, and it is much more limited: only works on 1D arrays, doesn't verify input sizes match, etc.  Most of the function is spent figuring out how to turn the block and grid indices and dimensions into unique offsets into the input arrays.  The pattern of computing a starting index and a stride is a common way to ensure that your grid size is independent of the input size.  The striding will maximize bandwidth by ensuring that threads with consecuitive indices are accessing consecutive memory locations as much as possible.  Thread indices beyond the length of the input (`x.shape[0]`, since `x` is a NumPy array) automatically skip over the for loop.

Also note that we did not need to specify a type signature for the CUDA kernel.  Unlike `@vectorize`, Numba can infer the type signature from the inputs automatically, and much more reliably.

Let's call the function now on some data:

In [5]:
import numpy as np

n = 100000
x = np.arange(n).astype(np.float32)
y = 2 * x
out = np.empty_like(x)

threads_per_block = 128
blocks_per_grid = 30

add_kernel[blocks_per_grid, threads_per_block](x, y, out)
print(out[:10])

[ 0.  3.  6.  9. 12. 15. 18. 21. 24. 27.]


The unusual syntax for calling the kernel function is designed to mimic the CUDA Runtime API in C, where the above call would look like:
```
add_kernel<<<blocks_per_grid, threads_per_block>>>(x, y, out)
```
The arguments within the square brackets define the size and shape of the thread grid, and the arguments with parentheses correspond to the kernel function arguments.

Note that, unlike the ufunc, the arguments are passed to the kernel as full NumPy arrays.  The kernel can access any element in the array it wants, regardless of its position in the thread grid.  This is why CUDA kernels are significantly more powerful that ufuncs.  (But with great power, comes a greater amount of typing...)

Numba includes [several helper functions](http://numba.pydata.org/numba-doc/dev/cuda/kernels.html#absolute-positions) to simplify the thread offset calculations above.  You can write the function much more simply as:

In [6]:
@cuda.jit
def add_kernel(x, y, out):
    start = cuda.grid(1)      # 1 = one dimensional thread grid, returns a single value
    stride = cuda.gridsize(1) # ditto

    # assuming x and y inputs are same length
    for i in range(start, x.shape[0], stride):
        out[i] = x[i] + y[i]

As before, using NumPy arrays forces Numba to allocate GPU memory, copy the arguments to the GPU, run the kernel, then copy the argument arrays back to the host.  This not very efficient, so you will often want to allocate device arrays:

In [7]:
x_device = cuda.to_device(x)
y_device = cuda.to_device(y)
out_device = cuda.device_array_like(x)

In [8]:
%timeit add_kernel[blocks_per_grid, threads_per_block](x, y, out)

1.49 ms ± 2.57 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [9]:
%timeit add_kernel[blocks_per_grid, threads_per_block](x_device, y_device, out_device); out_device.copy_to_host()

211 µs ± 1.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Kernel Synchronization

*One extremely important caveat should be mentioned here*: CUDA kernel execution is designed to be asynchronous with respect to the host program.  This means that the kernel launch (`add_kernel[blocks_per_grid, threads_per_block](x_device, y_device, out_device)`) returns immediately, allowing the CPU to continue executing while the GPU works in the background.  Only host<->device memory copies or an explicit synchronization call will force the CPU to wait until previously queued CUDA kernels are complete.

When you pass host NumPy arrays to a CUDA kernel, Numba has to synchronize on your behalf, but if you pass device arrays, processing will continue.  If you launch multiple kernels in sequence without any synchronization in between, they will be queued up to run sequentially by the driver, which is usually what you want.  If you want to run multiple kernels on the GPU in parallel (sometimes a good idea, but beware of race conditions!), take a look at [CUDA streams](http://numba.pydata.org/numba-doc/dev/cuda-reference/host.html?highlight=synchronize#stream-management).

Here's some sample timings (using `%time`, which only runs the statement once to ensure our measurement isn't affected by the finite depth of the CUDA kernel queue):

In [10]:
# CPU input/output arrays, implied synchronization for memory copies
%time add_kernel[blocks_per_grid, threads_per_block](x, y, out)

CPU times: user 3.33 ms, sys: 358 µs, total: 3.69 ms
Wall time: 3.01 ms


In [13]:
# GPU input/output arrays, no synchronization (but force sync before and after)
cuda.synchronize()
%time add_kernel[blocks_per_grid, threads_per_block](x_device, y_device, out_device)
cuda.synchronize()

CPU times: user 410 µs, sys: 44 µs, total: 454 µs
Wall time: 514 µs


In [14]:
# GPU input/output arrays, include explicit synchronization in timing
cuda.synchronize()
%time add_kernel[blocks_per_grid, threads_per_block](x_device, y_device, out_device); cuda.synchronize()

CPU times: user 334 µs, sys: 36 µs, total: 370 µs
Wall time: 375 µs


**Always be sure to synchronize with the GPU when benchmarking CUDA kernels!**

## Atomic Operations and Avoiding Race Conditions

CUDA, like many general purpose parallel execution frameworks, makes it possible to have race condtions in your code.  A race condition in CUDA arises when threads read or write a memory location that might be modified by another independent thread.  Generally speaking, you need to worry about:

 * read-after-write hazards: One thread is reading a memory location at the same time another thread might be writing to it.
 * write-after-write hazards: Two threads are writing to the same memory location, and only one write will be visible when the kernel is complete.
 
A common strategy to avoid both of these hazards is to organize your CUDA kernel algorithm such that each thread has exclusive responsibility for unique subsets of output array elements, and/or to never use the same array for both input and output in a single kernel call.  (Iterative algorithms can use a double-buffering strategy if needed, and switch input and output arrays on each iteration.)

However, there are many cases where different threads need to combine results.  Consider something very simple, like: "every thread increments a global counter."  Implementing this in your kernel requires each thread to:

1. Read the current value of a global counter.
2. Compute `counter + 1`.
3. Write that value back to global memory.

However, there is no guarantee that another thread has not changed the global counter between steps 1 and 3.  To resolve this problem, CUDA provides "atomic operations" which will read, modify and update a memory location in one, indivisible step.  Numba supports several of these functions, [described here](http://numba.pydata.org/numba-doc/dev/cuda/intrinsics.html#supported-atomic-operations).

Let's make our thread counter kernel:

In [15]:
@cuda.jit
def thread_counter_race_condition(global_counter):
    global_counter[0] += 1  # This is bad
    
@cuda.jit
def thread_counter_safe(global_counter):
    cuda.atomic.add(global_counter, 0, 1)  # Safely add 1 to offset 0 in global_counter array

In [17]:
# This gets the wrong answer
global_counter = cuda.to_device(np.array([0], dtype=np.int32))
thread_counter_race_condition[64, 64](global_counter)

print('Should be %d:' % (64*64), global_counter.copy_to_host())

Should be 4096: [1]


In [18]:
# This works correctly
global_counter = cuda.to_device(np.array([0], dtype=np.int32))
thread_counter_safe[64, 64](global_counter)

print('Should be %d:' % (64*64), global_counter.copy_to_host())

Should be 4096: [4096]


## Exercise

For this exercise, create a histogramming kernel.  This will take an array of input data, a range and a number of bins, and count how many of the input data elements land in each bin.  Below is an example CPU implementation of histogramming:

In [19]:
def cpu_histogram(x, xmin, xmax, histogram_out):
    '''Increment bin counts in histogram_out, given histogram range [xmin, xmax).'''
    # Note that we don't have to pass in nbins explicitly, because the size of histogram_out determines it
    nbins = histogram_out.shape[0]
    bin_width = (xmax - xmin) / nbins
    
    # This is a very slow way to do this with NumPy, but looks similar to what you will do on the GPU
    for element in x:
        bin_number = np.int32((element - xmin)/bin_width)
        if bin_number >= 0 and bin_number < histogram_out.shape[0]:
            # only increment if in range
            histogram_out[bin_number] += 1

In [79]:
x = np.random.normal(size=100000, loc=0, scale=1).astype(np.float32)
xmin = np.float32(-4.0)
xmax = np.float32(4.0)

In [83]:
%%timeit
histogram_out = np.zeros(shape=100, dtype=np.int32)
cpu_histogram(x, xmin, xmax, histogram_out)

697 ms ± 8.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [81]:
@cuda.jit
def cuda_histogram(x, xmin, xmax, histogram_out):
    '''Increment bin counts in histogram_out, given histogram range [xmin, xmax).'''
    start = cuda.grid(1)      # 1 = one dimensional thread grid, returns a single value
    stride = cuda.gridsize(1) # ditto
    
    nbins = histogram_out.shape[0]
    bin_width = (xmax - xmin) / nbins
    
    for i in range(start, x.shape[0], stride):
        bin_number = np.int32((x[i] - xmin)/bin_width)
        if bin_number >= 0 and bin_number < histogram_out.shape[0]:
            cuda.atomic.add(histogram_out,bin_number,1)
        

In [84]:
%%timeit
histogram_out = np.zeros(shape=100, dtype=np.int32)

d_x = cuda.to_device(x)
d_histogram_out = cuda.to_device(histogram_out)

cuda_histogram[10,32](d_x,xmin,xmax, d_histogram_out)

d_histogram_out.copy_to_host()

908 µs ± 1.31 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [3]:
from numba import cuda
import numpy as np

In [87]:
d_m = cuda.device_array([2048])

In [89]:
zeraMatrix[1,32](d_m)

In [76]:
resp = d_m.copy_to_host()

In [77]:
for i in resp:
    print(i)

0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
103.0
104.0
105.0
106.0
107.0
108.0
109.0
110.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
134.0
135.0
136.0
137.0
138.0
139.0
140.0
141.0
142.0
143.0
144.0
145.0
146.0
147.0
148.0
149.0
150.0
151.0
152.0
153.0
154.0
155.0
156.0
157.0
158.0
159.0
160.0
161.0
162.0
163.0
164.0
165.0
166.0
167.0
168.0
169.0
170.0
171.0
172.0
173.0
174.0
175.0
176.0
177.0
178.0
179.0
180.0
181.0
182.0
183.0
184.0


In [2]:
import numpy as np

In [1]:
import numba

In [71]:
@cuda.jit
def zeraMatrix(m):
    th_id = cuda.threadIdx.x
    b_id = cuda.blockIdx.x
        
    start = cuda.grid(1)      # 1 = one dimensional thread grid, returns a single value
    stride = cuda.gridsize(1)
    for i in range(start, m.shape[0],stride):
        m[i] = i
        print(th_id, b_id, i)

    

In [ ]:
x = cuda.threadIdx.x # this is the unique thread ID within a 1D block
    ty = cuda.blockIdx.x  # Similarly, this is the unique block ID within the 1D grid

    block_size = cuda.blockDim.x  # number of threads per block
    grid_size = cuda.gridDim.x    # number of blocks in the grid

In [31]:
from scipy.spatial import distance_matrix as dm

In [4]:
pontos = np.random.random([10000,2])

In [37]:
pontos = np.array(pontos,dtype =np.float32)

In [65]:
pontos

array([[0.7866952 , 0.52750202],
       [0.97538279, 0.78390795],
       [0.31142126, 0.43659442],
       ...,
       [0.40808955, 0.07633663],
       [0.91916746, 0.42480699],
       [0.00364194, 0.95989775]])

In [66]:
%%timeit
dm(pontos, pontos)

3.03 s ± 19.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [67]:
pontos.shape

(10000, 2)

In [44]:
import math

In [5]:
p = cuda.to_device(pontos)

In [6]:
resp = cuda.device_array([p.shape[0],p.shape[0]])

In [21]:
distance_matrix_cuda[3,32](p,resp)

CudaAPIError: [719] Call to cuLaunchKernel results in CUDA_ERROR_LAUNCH_FAILED

In [20]:
p.copy_to_host()

CudaAPIError: [719] Call to cuMemcpyDtoH results in CUDA_ERROR_LAUNCH_FAILED

In [55]:
testea = dm(pontos,pontos)

In [57]:
testeb = resp.copy_to_host()

CudaAPIError: [719] Call to cuMemcpyDtoH results in CUDA_ERROR_LAUNCH_FAILED

In [58]:
testea-testeb

array([[0.        , 0.56542027, 0.13738719, ..., 0.9328894 , 0.60257846,
        0.63960111],
       [0.56542027, 0.        , 0.60200936, ..., 0.8247593 , 0.48579034,
        0.58243352],
       [0.13738719, 0.60200936, 0.        , ..., 1.06298113, 0.72261035,
        0.76734143],
       ...,
       [0.9328894 , 0.8247593 , 1.06298113, ..., 0.        , 0.36605066,
        0.29748383],
       [0.60257846, 0.48579034, 0.72261035, ..., 0.36605066, 0.        ,
        0.0980288 ],
       [0.63960111, 0.58243352, 0.76734143, ..., 0.29748383, 0.0980288 ,
        0.        ]])

In [14]:
@cuda.jit
def distance_matrix_cuda(p,out):
    x,y = cuda.grid(2)
    stridex, stridey = cuda.gridDim.x, cuda.gridDim.y
    for i in range(x,out.shape[0],cuda.gridDim.x):
        for j in range(y,out.shape[1],cuda.gridDim.y):
            if(i<out.shape[0] and j<out.shape[1]):
                out[i][j] = math.sqrt((pontos[i][0]-pontos[j][0])**2 + (pontos[i][1]-pontos[j][1])**2)

In [6]:
@cuda.jit
def preenche(out):
    x,y = cuda.grid(2)
    stridex, stridey = cuda.gridDim.x, cuda.gridDim.y
    for i in range(x,out.shape[0],cuda.gridDim.x):
        for j in range(y,out.shape[1],cuda.gridDim.y):
            if(i<out.shape[0] and j<out.shape[1]):
                out[i][j] = i*out.shape[1] + j

In [13]:
import math

In [9]:
out = cuda.device_array([10,7])
preenche[1,10](out)
out.copy_to_host()

array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.],
       [ 7.,  8.,  9., 10., 11., 12., 13.],
       [14., 15., 16., 17., 18., 19., 20.],
       [21., 22., 23., 24., 25., 26., 27.],
       [28., 29., 30., 31., 32., 33., 34.],
       [35., 36., 37., 38., 39., 40., 41.],
       [42., 43., 44., 45., 46., 47., 48.],
       [49., 50., 51., 52., 53., 54., 55.],
       [56., 57., 58., 59., 60., 61., 62.],
       [63., 64., 65., 66., 67., 68., 69.]])

## 

In [162]:
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32

In [157]:
import math